In [286]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Konfiguration neuronale Netze

In [287]:
class NeuronalNetScalar(nn.Module):
        def __init__(self):
            super(NeuronalNetScalar, self).__init__()
            self.lin1 = nn.Linear(8,30)
            self.lin2 = nn.Linear(30,30)
            self.lin3 = nn.Linear(30,1)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            return x

# Datenbeschaffung

In [288]:
#Die Tore der einzelnen Mannschaften pro Spiel aus den matches in eine Liste schreiben
def inputValues(gameList, df_stats,inputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        input_values= df_stats[df_stats['HomeTeam']==homeTeam]
        input_values= input_values[input_values['AwayTeam']==awayTeam]
        input_values= input_values.reset_index(drop=True)
        input_values = input_values[:-1]
        inputList.append(input_values.loc[:,['FTHG','FTAG']])

In [289]:
#Methode um aus einer Liste eine Tensorliste mit Floatwerten generieren
def dfToTensor(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df.values).view(8)
        tensorList.append(tensor)

In [290]:
def dfToTensor_forOutput(dfList, tensorList, viewValue):
    for df in dfList:
        tensor = torch.FloatTensor(df).view(viewValue)
        tensorList.append(tensor)

# Output Scalar

In [308]:
#Outputvalues generieren, indem bei Heimsieg 0, bei unentschieden 1 und bei Auswärtssieg 2 gesetzt wird
def outputValuesScalar(gameList, df_stats,outputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        output_values= df_stats[df_stats['HomeTeam']==homeTeam]
        output_values= output_values[output_values['AwayTeam']==awayTeam]
        output_values= output_values.reset_index(drop=True)
        output_values = output_values.tail(1)
        output_values = output_values.loc[:,['FTHG','FTAG']]
        homegoals = output_values.loc[:,['FTHG']].iloc[0]['FTHG']
        awaygoals = output_values.loc[:,['FTAG']].iloc[0]['FTAG']
        if homegoals > awaygoals:
            df_out=[0.]
        if homegoals == awaygoals:
            df_out=[1.]
        if homegoals < awaygoals:
            df_out=[2.]
        outputList.append(df_out)

# Input Scalar

In [309]:
#Pfade Fabian
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv

#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv

In [293]:
def new_inputs_Scalar(path,TestInputListe,TestOutputListe):
    topTeams = pd.read_csv(path, sep=';')
    df = pd.DataFrame(topTeams, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
    df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')
    df_names = df_names[['HomeTeam','AwayTeam']]
    gameListe = []
    for row in df_names.itertuples():
        data = [[row.HomeTeam,row.AwayTeam]]
        gameListe.append(pd.DataFrame(data))
    inputValues(gameListe, df,TestInputListe)
    outputValuesScalar(gameListe,df,TestOutputListe)

In [294]:
inputList_0910_1314_Scalar = []
outputList_0910_1314_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv',inputList_0910_1314_Scalar,outputList_0910_1314_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv',inputList_0910_1314,outputList_0910_1314)

inputList_1011_1415_Scalar = []
outputList_1011_1415_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv',inputList_1011_1415_Scalar,outputList_1011_1415_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv',inputList_1011_1415,outputList_1011_1415)

inputList_1112_1516_Scalar = []
outputList_1112_1516_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv',inputList_1112_1516_Scalar,outputList_1112_1516_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv',inputList_1112_1516,outputList_1112_1516)

inputList_1213_1617_Scalar = []
outputList_1213_1617_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv',inputList_1213_1617_Scalar,outputList_1213_1617_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv',inputList_1213_1617,outputList_1213_1617)

inputList_1314_1718_Scalar = []
outputList_1314_1718_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv',inputList_1314_1718_Scalar,outputList_1314_1718_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv',inputList_1314_1718,outputList_1314_1718)

inputList_1415_1819_Scalar = []
outputList_1415_1819_Scalar = []
new_inputs_Scalar('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv',inputList_1415_1819_Scalar,outputList_1415_1819_Scalar)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv',inputList_1415_1819,outputList_1415_1819)

In [295]:
#In Liste inputList_1415_1819 waren leere Dataframes drin, die werden in diesem Zwischenschritt rausgelöscht
for i in range(len(inputList_1415_1819_Scalar)):
    if inputList_1415_1819_Scalar[i].empty:
        print(i)

11
14
32
46
54
63
85
92
97
109
116


In [296]:
inputList_1415_1819_Scalar.pop(11)
inputList_1415_1819_Scalar.pop(13)
inputList_1415_1819_Scalar.pop(30)
inputList_1415_1819_Scalar.pop(43)
inputList_1415_1819_Scalar.pop(50)
inputList_1415_1819_Scalar.pop(58)
inputList_1415_1819_Scalar.pop(79)
inputList_1415_1819_Scalar.pop(85)
inputList_1415_1819_Scalar.pop(89)
inputList_1415_1819_Scalar.pop(100)
inputList_1415_1819_Scalar.pop(106)

outputList_1415_1819_Scalar.pop(11)
outputList_1415_1819_Scalar.pop(13)
outputList_1415_1819_Scalar.pop(30)
outputList_1415_1819_Scalar.pop(43)
outputList_1415_1819_Scalar.pop(50)
outputList_1415_1819_Scalar.pop(58)
outputList_1415_1819_Scalar.pop(79)
outputList_1415_1819_Scalar.pop(85)
outputList_1415_1819_Scalar.pop(89)
outputList_1415_1819_Scalar.pop(100)
outputList_1415_1819_Scalar.pop(106)

print(len(inputList_1415_1819_Scalar))
print(len(outputList_1415_1819_Scalar))

110
110


In [297]:
train_tensor_input_scalar = []
dfToTensor(inputList_0910_1314_Scalar,train_tensor_input_scalar)
dfToTensor(inputList_1011_1415_Scalar,train_tensor_input_scalar)
dfToTensor(inputList_1112_1516_Scalar,train_tensor_input_scalar)
dfToTensor(inputList_1213_1617_Scalar,train_tensor_input_scalar)
dfToTensor(inputList_1314_1718_Scalar,train_tensor_input_scalar)

test_tensor_input_scalar = []
dfToTensor(inputList_1415_1819_Scalar,test_tensor_input_scalar)

In [298]:
train_tensor_target_scalar = []
dfToTensor_forOutput(outputList_0910_1314_Scalar,train_tensor_target_scalar,1)
dfToTensor_forOutput(outputList_1011_1415_Scalar,train_tensor_target_scalar,1)
dfToTensor_forOutput(outputList_1112_1516_Scalar,train_tensor_target_scalar,1)
dfToTensor_forOutput(outputList_1213_1617_Scalar,train_tensor_target_scalar,1)
dfToTensor_forOutput(outputList_1314_1718_Scalar,train_tensor_target_scalar,1)

test_tensor_target_scalar = []
dfToTensor_forOutput(outputList_1415_1819_Scalar,test_tensor_target_scalar,1)

In [310]:
print(train_tensor_input_scalar[1])
print(train_tensor_target_scalar[1])
print(test_tensor_input_scalar[1])
print(test_tensor_target_scalar[1])

tensor([4., 0., 2., 0., 0., 0., 0., 0.])
tensor([0.])
tensor([2., 2., 2., 1., 2., 2., 0., 3.])
tensor([2.])


# Trainieren des neuronalen Netzes Skalar

In [311]:
net = NeuronalNetScalar()

In [312]:
criterion = nn.L1Loss()

In [313]:
optimizer = torch.optim.SGD(net.parameters(),lr =0.01, momentum = 0.9)

In [315]:
for epoch in range(50):
    running_loss = 0.0
    for i in range(len(train_tensor_target_scalar)):
        optimizer.zero_grad()
        outputs = net(train_tensor_input_scalar[i])
        loss = criterion(outputs, train_tensor_target_scalar[i])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 305 == 304:
            print (epoch + 1 , i+1 , running_loss / 305)
            running_loss = 0.0

1 305 0.8426229508196721
1 610 0.8032786885245902
2 305 0.8426229508196721
2 610 0.8032786885245902
3 305 0.8426229508196721
3 610 0.8032786885245902
4 305 0.8426229508196721
4 610 0.8032786885245902
5 305 0.8426229508196721
5 610 0.8032786885245902
6 305 0.8426229508196721
6 610 0.8032786885245902
7 305 0.8426229508196721
7 610 0.8032786885245902
8 305 0.8426229508196721
8 610 0.8032786885245902
9 305 0.8426229508196721
9 610 0.8032786885245902
10 305 0.8426229508196721
10 610 0.8032786885245902
11 305 0.8426229508196721
11 610 0.8032786885245902
12 305 0.8426229508196721
12 610 0.8032786885245902
13 305 0.8426229508196721
13 610 0.8032786885245902
14 305 0.8426229508196721
14 610 0.8032786885245902
15 305 0.8426229508196721
15 610 0.8032786885245902
16 305 0.8426229508196721
16 610 0.8032786885245902
17 305 0.8426229508196721
17 610 0.8032786885245902
18 305 0.8426229508196721
18 610 0.8032786885245902
19 305 0.8426229508196721
19 610 0.8032786885245902
20 305 0.8426229508196721
20 6

In [304]:
#Quantitative Evaluierung 
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_tensor_target_scalar)) :
        outputs = net(test_tensor_input_scalar[i])
        if outputs < 0.5:
            predicted = 0
        elif outputs <1.5:
            predicted = 1
        else:
            predicted = 2
        total += test_tensor_target_scalar[i].size(0)
        correct += (predicted == test_tensor_target_scalar[i]).sum().item()
print(correct / len(test_ten))

Accuracy: 0 %


In [305]:
output = net(test_tensor_input[1])
print(output)

tensor([0.], grad_fn=<ReluBackward0>)


In [306]:
net(test_tensor_input_scalar[7])

tensor([0.], grad_fn=<ReluBackward0>)

In [307]:
train_tensor_input_scalar[2]

tensor([0., 0., 0., 3., 0., 0., 2., 2.])

In [235]:
train_tensor_target_scalar[2]

tensor([2.])